In [ ]:
!pip install sentence-transformers
!pip install faiss-cpu
!pip install transformers
!pip install gradio
!pip install feedparser
!pip install streamlit
!pip install PyMuPDF
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [ ]:
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import gradio as gr
import streamlit as st
from tqdm import tqdm

In [ ]:
import feedparser

def search_arxiv(query, max_results=20):
    from urllib.parse import quote
    base_url = 'http://export.arxiv.org/api/query?'
    encoded_query = quote(f'all:{query}')
    search_query = f'search_query={encoded_query}&start=0&max_results={max_results}'
    feed = feedparser.parse(base_url + search_query)

    papers = []
    for entry in feed.entries:
        title = entry.title.strip().replace('\n', ' ')
        summary = entry.summary.strip().replace('\n', ' ')
        authors = [author.name for author in entry.authors] if hasattr(entry, 'authors') else []
        published = entry.published if hasattr(entry, 'published') else "Unknown"
        link = entry.link if hasattr(entry, 'link') else ""
        papers.append({
            'title': title,
            'abstract': summary,
            'authors': authors,
            'published': published,
            'link': link
        })
    return papers



In [ ]:
from transformers import pipeline

# Load summarization pipeline
summarizer = pipeline("summarization", model="t5-base")
print("✅ Summarizer model loaded!")

# Define summarizer function
def summarize_input_text(text, chunk_size=512):
    if not text or len(text.strip()) < 20:
        return "No summary available."

    # Clean and split
    words = text.replace("\n", " ").replace("-", " ").strip().split()

    # Chunking into multiple pieces
    chunks = [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

    summaries = []
    for chunk in chunks:
        summary = summarizer(
            chunk,
            max_length=200,
            min_length=30,
            do_sample=False
        )
        summaries.append(summary[0]['summary_text'])

    return " ".join(summaries)





/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cuda:0


✅ Summarizer model loaded!


In [ ]:
domain_structure = {
    "Artificial Intelligence": [
        "Machine Learning", "Deep Learning", "Reinforcement Learning", "Transfer Learning",
        "Self-Supervised Learning", "Meta-Learning", "Federated Learning", "Explainable AI",
        "Generative AI", "AI Ethics"
    ],
    "Healthcare & Biomedical AI": [
        "Medical Imaging", "Clinical NLP", "Drug Discovery", "Disease Diagnosis Models",
        "Predictive Healthcare", "Public Health Analytics", "Electronic Health Records", "Wearable Health Devices",
        "Mental Health AI", "COVID-19 Research"
    ],
    "Finance & Economics": [
        "Stock Price Prediction", "Fraud Detection", "Algorithmic Trading", "Credit Risk Modeling",
        "Financial Sentiment Analysis", "AI in Insurance", "Economic Forecasting", "Portfolio Optimization",
        "Blockchain & Crypto Analytics", "Regulatory Technology"
    ],
    "Law & Policy": [
        "Legal Document Summarization", "AI in Legal Reasoning", "Contract Analysis & NLP", "Legal QA",
        "Case Law Retrieval", "Judgment Prediction", "Legal Chatbots", "Privacy Law Compliance",
        "Policy Modeling", "AI in Legal Reasoning"
    ],
    "Education & Social Science": [
        "Intelligent Tutoring Systems", "AI in Education", "Student Performance Prediction", "Academic Plagiarism Detection",
        "Exam Question Generation", "Adaptive Learning", "Education Data Mining", "Misinformation in EdTech",
        "Educational NLP", "Conversational Agents in Education"
    ],
    "Sustainability, Industry & Robotics": [
        "Climate Modeling", "Renewable Energy", "Smart Grids", "AI in Agriculture",
        "Precision Farming", "Autonomous Vehicles", "Robotics and Automation", "Predictive Maintenance",
        "Industrial Automation", "Disaster Response"
    ]
}

#all_papers = []
#for domain, subdomains in domain_structure.items():
#    for sub in subdomains:
#        print(f"Fetching papers for: {domain} → {sub}")
#        papers = search_arxiv(sub, max_results=20)
#        for paper in papers:
#            paper['domain'] = domain
#            paper['subdomain'] = sub
#        all_papers.extend(papers)

#print(f"✅ Total collected papers: {len(all_papers)}")
#df_all = pd.DataFrame(all_papers)
#def batch_summarize(texts, batch_size=16):
#    summaries = []
#    num_batches = len(texts) // batch_size + int(len(texts) % batch_size != 0)
#    for i in tqdm(range(0, len(texts), batch_size),desc="🔄 Summarizing batches", unit="batch"):
#        batch = texts[i:i+batch_size]
#        max_tokens = max(30, int(len(batch[0].split()) * 0.6))
#        result = summarizer(batch, max_new_tokens=max_tokens, min_length=20,do_sample=False)
#        summaries.extend([r['summary_text'] for r in result])
#    return summaries

# 🔁 Replace per-row apply with batched summarization
#df_all['summary'] = batch_summarize(df_all['abstract'].tolist())

# Save the new DataFrame
#df_all.to_csv("arxiv_1200_papers.csv", index=False)
#print("✅ Summarization complete and saved to CSV!")

In [ ]:
# Check paper counts per subdomain
#subdomain_counts = df_all.groupby(['domain', 'subdomain']).size().reset_index(name='num_papers')

# Display sorted counts
#print(subdomain_counts.sort_values('num_papers'))


In [ ]:
import pandas as pd

df_loaded = pd.read_csv("sample_data/arxiv_1200_papers.csv")
print(f"✅ Loaded CSV with {df_loaded.shape[0]} papers.")
df_loaded.head()


✅ Loaded CSV with 1200 papers.


,title,abstract,authors,published,link,domain,subdomain,summary
0,Lecture Notes: Optimization for Machine Learning,Lecture notes on optimization for machine lear...,['Elad Hazan'],2019-09-08T21:49:42Z,http://arxiv.org/abs/1909.03550v1,Artificial Intelligence,Machine Learning,lecture notes on optimization for machine lear...
1,An Optimal Control View of Adversarial Machine...,I describe an optimal control view of adversar...,['Xiaojin Zhu'],2018-11-11T14:28:34Z,http://arxiv.org/abs/1811.04422v1,Artificial Intelligence,Machine Learning,this view encompasses many types of adversaria...
2,Minimax deviation strategies for machine learn...,The article is devoted to the problem of small...,"['Michail Schlesinger', 'Evgeniy Vodolazskiy']",2017-07-16T09:15:08Z,http://arxiv.org/abs/1707.04849v1,Artificial Intelligence,Machine Learning,the article is devoted to the problem of small...
3,Machine Learning for Clinical Predictive Analy...,"In this chapter, we provide a brief overview o...",['Wei-Hung Weng'],2019-09-19T22:02:00Z,http://arxiv.org/abs/1909.09246v1,Artificial Intelligence,Machine Learning,"in this chapter, we provide a brief overview o..."
4,Towards Modular Machine Learning Solution Deve...,Machine learning technologies have demonstrate...,"['Samiyuru Menik', 'Lakshmish Ramaswamy']",2023-01-23T22:54:34Z,http://arxiv.org/abs/2301.09753v1,Artificial Intelligence,Machine Learning,cost of developing custom machine learning sol...


In [ ]:
embed_model = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ Embedding model loaded.")

abstract_texts = df_loaded['summary'].fillna("").tolist()
abstract_embeddings = embed_model.encode(abstract_texts, show_progress_bar=True)
abstract_embeddings = np.array(abstract_embeddings).astype('float32')

dimension = abstract_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(abstract_embeddings)
print(f"✅ FAISS index created with {index.ntotal} papers!")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Embedding model loaded.


Batches:   0%|          | 0/38 [00:00<?, ?it/s]

✅ FAISS index created with 1200 papers!


In [ ]:
from transformers import pipeline

# Load zero-shot classification pipeline
classifier = pipeline(
    "zero-shot-classification",
    model="t5-base"
)
print("✅ Zero-shot classifier loaded!")
candidate_labels = [
    # AI
    "Machine Learning", "Deep Learning", "Reinforcement Learning", "Transfer Learning",
    "Self-Supervised Learning", "Meta-Learning", "Federated Learning", "Explainable AI",
    "Generative AI", "AI Ethics",

    # Healthcare
    "Medical Imaging", "Clinical NLP", "Drug Discovery", "Disease Diagnosis Models",
    "Predictive Healthcare", "Public Health Analytics", "Electronic Health Records",
    "Wearable Health Devices", "Mental Health AI", "COVID-19 Research",

    # Finance
    "Stock Price Prediction", "Fraud Detection", "Algorithmic Trading", "Credit Risk Modeling",
    "Financial Sentiment Analysis", "AI in Insurance", "Economic Forecasting",
    "Portfolio Optimization", "Blockchain & Crypto Analytics", "Regulatory Technology",

    # Law
    "Legal Document Summarization", "AI in Legal Reasoning", "Contract Analysis & NLP",
    "Legal QA", "Case Law Retrieval", "Judgment Prediction", "Legal Chatbots",
    "Privacy Law Compliance", "Policy Modeling",

    # Education
    "Intelligent Tutoring Systems", "AI in Education", "Student Performance Prediction",
    "Academic Plagiarism Detection", "Exam Question Generation", "Adaptive Learning",
    "Education Data Mining", "Misinformation in EdTech", "Educational NLP",
    "Conversational Agents in Education",

    # Sustainability & Robotics
    "Climate Modeling", "Renewable Energy", "Smart Grids", "AI in Agriculture",
    "Precision Farming", "Autonomous Vehicles", "Robotics & Control",
    "Predictive Maintenance", "Industrial Automation", "Disaster Response"
]

# Define classification function
def classify_domain(abstract):
    result = classifier(abstract, candidate_labels)
    return result['labels'][0]


Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at t5-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.


✅ Zero-shot classifier loaded!


In [ ]:
def find_related_papers(user_input, top_k=5, domain_filter="All", subdomain_filter="All"):
    filtered_df = df_loaded.copy()
    if domain_filter != "All":
        filtered_df = filtered_df[filtered_df['domain'] == domain_filter]
    if subdomain_filter != "All":
        filtered_df = filtered_df[filtered_df['subdomain'] == subdomain_filter]

    # Embed user input
    user_embedding = embed_model.encode([user_input]).astype('float32')

    # Embed abstracts from filtered data
    abstracts = filtered_df['abstract'].tolist()
    embeddings = embed_model.encode(abstracts).astype('float32')

    # Build dynamic FAISS index
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)

    # Search for top_k matches
    distances, indices = index.search(user_embedding, min(top_k, len(filtered_df)))

    results = filtered_df.iloc[indices[0]].copy()
    results['similarity_score'] = distances[0]
    results['summary'] = results['abstract'].apply(summarize_input_text)

    return results[results['similarity_score'] < 1.0]


In [ ]:
def generate_csv(df):
    df_out = df[['title', 'summary', 'link', 'domain', 'subdomain', 'similarity_score']]
    df_out.to_csv("recommendations.csv", index=False)
    return "recommendations.csv"

In [ ]:
def summarizer_and_recommender(user_input):
    if not user_input.strip():
        return "⚠️ Please enter some input text."

    summary = summarize_input_text(user_input)
    recommendations = find_related_papers(user_input)

    output = f"### ✅ Your Input Summary:\n{summary}\n\n"
    output += "### 📚 Recommended Papers:\n\n"

    for _, row in recommendations.iterrows():
      output += f"**Title:** {row['title']}  \n"

      if 'authors' in row and isinstance(row['authors'], str):
        try:
            authors = eval(row['authors']) if row['authors'].startswith("[") else [row['authors']]
        except:
            authors = [row['authors']]
        output += f"**Authors:** {', '.join(authors)}  \n"

      if 'published' in row:
        output += f"**Published:** {row['published']}  \n"

      if 'link' in row:
        output += f"**Link:** [View Paper]({row['link']})  \n"

      output += f"**Domain:** {row['domain']}  \n"
      output += f"**Similarity Score:** {row['similarity_score']:.2f}  \n"
      output += f"**Summary:** {row['summary']}  \n"
      output += "---  \n\n"


    return output




In [ ]:
# Define your input text (replace with anything you want to test)
user_input_text = """
Recent advances in self-supervised learning have enabled models to learn powerful representations from large-scale unlabeled data. In the vision domain, methods such as SimCLR, MoCo, and DINO have shown that contrastive and non-contrastive objectives can yield robust visual features. At the same time, transformer-based language models like BERT and RoBERTa have demonstrated the effectiveness of masked language modeling for text representation learning. These breakthroughs have motivated the development of unified models that learn from both image and text modalities using self-supervised objectives.

In this work, we introduce UniVL, a unified vision-language model trained entirely via self-supervised learning. UniVL combines masked image modeling, masked language modeling, and cross-modal contrastive learning into a single pretraining framework. The model uses a dual-encoder architecture with separate transformers for vision and language, along with a fusion module for cross-modal alignment. We train UniVL on a large-scale web-crawled dataset of image-caption pairs, enabling the model to learn joint representations that capture both semantic alignment and modality-specific structures.

We evaluate UniVL on a range of downstream tasks including image-text retrieval, visual question answering, and image captioning. Our results show that UniVL significantly outperforms prior self-supervised baselines and even rivals fully supervised methods. Notably, we find that combining masked modeling with contrastive learning is key to unlocking the full potential of multi-modal self-supervision. Ablation studies confirm that each objective contributes complementary strengths, and the joint training setup promotes better generalization across tasks.

This study highlights the power of self-supervised learning as a scalable and versatile approach for vision-language modeling. By eliminating the need for costly annotations and exploiting the natural co-occurrence of image and text data, UniVL demonstrates that strong multi-modal representations can emerge from raw signals alone. Our findings suggest a promising future for self-supervised pretraining in multimodal AI, with potential applications in search, accessibility, and human-computer interaction.

"""

print("✅ USER INPUT TEXT")
print(user_input_text)
print("\n==============================\n")

# 1️⃣ Summarize the input text
user_summary = summarize_input_text(user_input_text)
print("✅ SUMMARIZED INPUT")
print(user_summary)
print("\n==============================\n")

# 2️⃣ Find related papers
recommendations = find_related_papers(user_input_text, top_k=5)

print("✅ RECOMMENDED PAPERS")
for idx, row in recommendations.iterrows():
    print(f"- Title: {row['title']}")
    print(f"  Authors: {', '.join(eval(row['authors'])) if isinstance(row['authors'], str) else row['authors']}")
    print(f"  Published: {row['published']}")
    print(f"  Link: {row['link']}")
    print(f"  Domain: {row['domain']}")
    print(f"  Summary: {row['summary']}")
    print(f"  Similarity Score: {row['similarity_score']:.2f}")
    print("--------------------------------")


✅ USER INPUT TEXT

Recent advances in self-supervised learning have enabled models to learn powerful representations from large-scale unlabeled data. In the vision domain, methods such as SimCLR, MoCo, and DINO have shown that contrastive and non-contrastive objectives can yield robust visual features. At the same time, transformer-based language models like BERT and RoBERTa have demonstrated the effectiveness of masked language modeling for text representation learning. These breakthroughs have motivated the development of unified models that learn from both image and text modalities using self-supervised objectives.

In this work, we introduce UniVL, a unified vision-language model trained entirely via self-supervised learning. UniVL combines masked image modeling, masked language modeling, and cross-modal contrastive learning into a single pretraining framework. The model uses a dual-encoder architecture with separate transformers for vision and language, along with a fusion module 

Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


✅ SUMMARIZED INPUT
recent advances in self supervised learning have enabled models to learn powerful representations from large scale unlabeled data . in the vision domain, methods such as SimCLR, MoCo, and DINO have shown that contrastive and non contrastive objectives can yield robust visual features .




Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


✅ RECOMMENDED PAPERS
- Title: Fast, Effective, and Self-Supervised: Transforming Masked Language   Models into Universal Lexical and Sentence Encoders
  Authors: Fangyu Liu, Ivan Vulić, Anna Korhonen, Nigel Collier
  Published: 2021-04-16T10:49:56Z
  Link: http://arxiv.org/abs/2104.08027v2
  Domain: Artificial Intelligence
  Summary: pretrained Masked Language Models (MLMs) have revolutionised NLP in recent years . previous work indicated off the shelf MLMs are not effective as universal lexical or sentence encoders . we propose an extremely simple, fast and effective contrastive learning technique . mirror BERT relies on fully identical or slightly modified string pairs .
  Similarity Score: 0.58
--------------------------------
- Title: Synthetic Misinformers: Generating and Combating Multimodal   Misinformation
  Authors: Stefanos-Iordanis Papadopoulos, Christos Koutlis, Symeon Papadopoulos, Panagiotis C. Petrantonakis
  Published: 2023-03-02T12:59:01Z
  Link: http://arxiv.org/abs/2

In [ ]:
def format_apa_citation(title, authors, published, link):
    try:
        year = published.split("-")[0]
    except:
        year = "n.d."

    if isinstance(authors, str):
        try:
            authors_list = eval(authors)
        except:
            authors_list = [authors]
    else:
        authors_list = authors

    author_str = ", ".join(authors_list[:3])
    if len(authors_list) > 3:
        author_str += ", et al."

    return f"{author_str} ({year}). <i>{title}</i>. Retrieved from <a href='{link}' target='_blank'>{link}</a>"

In [ ]:

all_subdomains = sorted(set(sd for subs in domain_structure.values() for sd in subs))


def summarizer_and_recommender_ui(user_input, domain, subdomain, history):
    if not user_input.strip():
        return "⚠️ Please enter some input text.", display_history(history), history

    history.append(user_input.strip())

    summary = summarize_input_text(user_input)
    recommendations = find_related_papers(user_input, domain_filter=domain, subdomain_filter=subdomain)
    csv_file_path = generate_csv(recommendations)


    # Format output (HTML)
    output = f"<h4>✅ Your Input Summary</h4><p>{summary}</p><br><h4>📚 Recommended Papers</h4><br>"

    for _, row in recommendations.iterrows():
        citation = format_apa_citation(row['title'], row['authors'], row['published'], row['link'])
        output += f"<details><summary>📚 APA Citation</summary><p style='font-size:small'>{citation}</p></details>"
        output += f"<p><b>Title:</b> {row['title']}<br>"
        authors = eval(row['authors']) if isinstance(row['authors'], str) and row['authors'].startswith("[") else [row['authors']]
        output += f"<b>Authors:</b> {', '.join(authors)}<br>"
        output += f"<b>Published:</b> {row['published']}<br>"
        output += f"<b>Link:</b> <a href='{row['link']}' target='_blank'>View Paper</a><br>"
        output += f"<b>Domain:</b> {row['domain']}<br><b>Subdomain:</b> {row.get('subdomain', 'N/A')}<br>"
        output += f"<b>Similarity Score:</b> {row['similarity_score']:.2f}<br>"
        output += f"<b>Summary:</b> {row['summary']}</p><hr><br>"


    return output, display_history(history), history, csv_file_path

def display_history(history_list):
    if not history_list:
        return "<i>No past topics yet.</i>"
    return "<h4>🕘 Past Topics Searched</h4><ul>" + "".join([f"<li>{item}</li>" for item in reversed(history_list)]) + "</ul>"

def update_subdomains(selected_domain):
    if selected_domain == "All":
        return gr.update(choices=["All"] + all_subdomains, value="All")
    return gr.update(choices=["All"] + domain_structure[selected_domain], value="All")


with gr.Blocks(title="AI Research Paper Summarizer & Recommender") as demo:
    gr.Markdown("<h2 style='text-align: center;'>🧠 Research Paper Summarizer & Recommender</h2>")

    with gr.Row():
        user_input = gr.Textbox(label="Enter your research topic or abstract", lines=10)

    with gr.Row():
        domain_dropdown = gr.Dropdown(
            label="Select Domain",
            choices=["All"] + list(domain_structure.keys()),
            value="All"
        )
        subdomain_dropdown = gr.Dropdown(
            label="Select Subdomain",
            choices=["All"] + all_subdomains,
            value="All"
        )

    domain_dropdown.change(fn=update_subdomains, inputs=domain_dropdown, outputs=subdomain_dropdown)

    history_state = gr.State([])

    with gr.Row():
        submit_button = gr.Button("Get Summary & Recommendations")

    summary_output = gr.HTML()
    history_output = gr.HTML()
    csv_output = gr.File(label="📥 Download Paper Recommendations")

    submit_button.click(
        fn=summarizer_and_recommender_ui,
        inputs=[user_input, domain_dropdown, subdomain_dropdown, history_state],
        outputs=[summary_output, history_output, history_state, csv_output]
    )

demo.launch(share=True)





Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5041c6c1970952cdad.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
